In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as w
from IPython.display import display
import io
from ipywidgets import HBox, VBox

s = w.FloatRangeSlider(min=0,max=100,step=1,value=[0,10])
s_hb = HBox([w.Label(value="Seedling:"),s])

v = w.FloatRangeSlider(
            min=0,
            max=100,
            step=1,
            value=[10,20]
        )
v_hb = HBox([
        w.Label(value="Vegetative:"),
        v]
)

p = w.FloatRangeSlider(
            min=0,
            max=100,
            step=1,
            value=[20,30]
        )
p_hb = HBox([
        w.Label(value="Preflowering:"),
        p]
)
f=w.FloatRangeSlider(
            min=0,
            max=100,
            step=1,
            value=[30,40]
        )
f_hb = HBox([
        w.Label(value="Flowering:"),
        f]
)

hbox0 = HBox([s_hb,v_hb, p_hb, f_hb])

temp1 = w.IntSlider(
    min=0,
    max=200,
    step=1,
    value=100
)
temp1_hb = HBox([w.Label(value="Stage 1 Norm:"), temp1])
temp2 = w.IntSlider(
    min=0,
    max=200,
    step=1,
    value=100
)
temp2_hb=HBox([w.Label(value="Stage 2 Norm:"), temp2])
temp3 = w.IntSlider(
    min=0,
    max=200,
    step=1,
    value=100
)
temp3_hb = HBox([w.Label(value="Stage 3 Norm:"), temp3])
temp4=w.IntSlider(
    min=0,
    max=200,
    step=1,
    value=100
)
temp4_hb = HBox([w.Label(value="Stage 4 Norm:"), temp4])
temp5=w.IntSlider(
    min=0,
    max=200,
    step=1,
    value=100
)
temp5_hb = HBox([w.Label(value="Stage 5 Norm:"), temp5])
hbox1 = HBox([temp1_hb, temp2_hb, temp3_hb])
hbox2 = HBox([temp4_hb, temp5_hb])


plt.rcParams["figure.figsize"] = [10, 5]
plt.rcParams["figure.autolayout"] = True

upload = w.FileUpload(
    accept='.csv',
    multiple=False
)
display(upload)

button = w.Button(description="Submit")
output = w.Output()
cont_button = w.Button(description="Continue")

def on_value_change(change):
    with upload and output:
        input_file = list(upload.value.values())[0]
        content = input_file['content']
        content = io.StringIO(content.decode('utf-8'))
        
        global df
        df = pd.read_csv(content)
        columns_list = []
        for col in df.columns:
            columns_list.append(col)
        
        global x_column
        x_column = w.Dropdown(
            options=columns_list,
            value=columns_list[0],
            description='X Axis:',
            disabled=False,
        )

        global y_column
        y_column = w.Dropdown(
            options=columns_list,
            value=columns_list[1],
            description='Y Axis:',
            disabled=False,
        )
        dropdown_boxes = HBox([x_column,y_column])
        
        display(dropdown_boxes)
        display(cont_button)
        
        
def axis_set(b):
    with output:
        print(" ")
        x = df[x_column.value]
        y = df[y_column.value]
        s.max = x.size-1
        v.max = x.size-1
        p.max = x.size-1
        f.max = x.size-1

        s.value = [0, round(x.size/4)]
        v.value = [round(x.size/4), round(x.size/2)]
        p.value = [round(x.size/2),round((3*x.size)/4)]
        f.value = [round((3*x.size)/4), x.size-1]

        temp1.max = temp2.max = temp3.max = temp4.max = temp5.max = y.max()
        temp1.value = temp2.value = temp3.value = temp4.value = temp5.value = y.mean() 

    
        display(hbox0)
        display(hbox1)
        display(hbox2)
        display(button)

        
display(output)
upload.observe(on_value_change, names='value')

def save_figure(b):
    with output:
        fig1.savefig("CantrakPlot.png",dpi=100)
                
def upload_complete(b):
    with output:
        print("Producing Plot:")
        input_file = list(upload.value.values())[0]
        content = input_file['content']
        content = io.StringIO(content.decode('utf-8'))
        
        df = pd.read_csv(content)
        x = df[x_column.value]
        y = df[y_column.value]

        o1, o2, o3, o4, o5 = temp1.value, temp2.value, temp3.value, temp4.value, temp5.value

        h1 = 0
        h2 = s.value[1]
        h3 = v.value[1]
        h4 = p.value[1]
        h5 = f.value[1]
        
        
        fig, ax = plt.subplots()
   
        plt.plot(x, y, label=y_column.value)
    
        # Set ticks

        ax.set_xticks(ticks=[x[0],x[round(x.size/4)],x[round(x.size/2)], x[round((3*x.size)/4)],x[x.size-1]])

                #PLOTTING 
        plt.axvline(x=h2, ymin=0, color='red', linestyle='dotted', linewidth=2, label="Seedling Phase")
        plt.axvline(x=h3, ymin=0, color='blue', linestyle='dotted', linewidth=2, label="Vegetating Phase")
        plt.axvline(x=h4, ymin=0, color='orange', linestyle='dotted', linewidth=2, label="Pre-flowering Phase")
        plt.axvline(x=h5, ymin=0, color='green', linestyle='dotted', linewidth=2, label="Flowering Phase")
        
        l1 = plt.hlines(y=o1, xmin=x[h1], xmax=x[h2], color='red', label="Norm")
        plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
        l2 = plt.hlines(y=o2, xmin=x[h2], xmax=x[h3], color='red')
        l3 = plt.hlines(y=o3, xmin=x[h3], xmax=x[h4], color='red')
        l4 = plt.hlines(y=o4, xmin=x[h4], xmax=x[h5], color='red')
        l5 = plt.hlines(y=o5, xmin=x[h5], xmax=df.tail(1)['time'], color='red')

        plt.xlim(x[0], x[x.size-1])
        global fig1
        fig1 = plt.gcf()
        
        plt.show()
        button2 = w.Button(description="Download Plot")
        display(button2)
        
        button2.on_click(save_figure)

button.on_click(upload_complete)
cont_button.on_click(axis_set)


FileUpload(value={}, accept='.csv', description='Upload')

Output()